## Problem 1

In [8]:
def extract_data_from_line(line):
    return list(map(lambda vect: (int(vect[0]), int(vect[1])), map(lambda x: x.split(','), line[2:].split(' v='))))

def read_input(filename):
    f = open(f'../inputs/{filename}.txt', 'r')

    robots = []
    while True:
        line = f.readline()
        if line == '':
            break
        [pos, vel] = extract_data_from_line(line)
        robots.append((pos, vel))

    f.close()
    return robots

In [135]:
# quadrant numbering:
# 0 1
# 2 3
def quadrant(pos, room_size):
    if pos[0] < (room_size[0]-1)/2 and pos[1] < (room_size[1]-1)/2:
        return 0
    if pos[0] > (room_size[0]-1)/2 and pos[1] < (room_size[1]-1)/2:
        return 1
    if pos[0] < (room_size[0]-1)/2 and pos[1] > (room_size[1]-1)/2:
        return 2
    if pos[0] > (room_size[0]-1)/2 and pos[1] > (room_size[1]-1)/2:
        return 3
    return None

def new_position(pos, vel, seconds, room_size):
    res = []
    for i in range(2):
        res.append((pos[i] + vel[i] * seconds) % room_size[i])
    return (res[0], res[1])

def solve1(input_filename, room_size, seconds):
    robots = read_input(input_filename)
    new_positions = list(map(lambda r: new_position(r[0], r[1], seconds, room_size), robots))
    print(new_positions)
    quad = [0, 0, 0, 0]
    for pos in new_positions:
        quad_pos = quadrant(pos, room_size)
        if quad_pos != None:
            quad[quad_pos] += 1

    return quad[0]*quad[1]*quad[2]*quad[3]

## Problem 2

In [72]:
import matplotlib.pyplot as plt

In [11]:
def read_input2(filename):
    f = open(f'../inputs/{filename}.txt', 'r')

    xlist = []
    ylist = []
    velocities = []
    while True:
        line = f.readline()
        if line == '':
            break
        [pos, vel] = extract_data_from_line(line)
        xlist.append(pos[0])
        ylist.append(pos[1])
        velocities.append(vel)

    f.close()
    return (xlist, ylist, velocities)

In [143]:
def positions_after_seconds(xlist, ylist, velocities, seconds, room_size):
    new_xlist, new_ylist = [], []
    for i in range(len(xlist)):
        new_xlist.append((xlist[i] + velocities[i][0] * seconds) % room_size[0])
    for j in range(len(ylist)):
        new_ylist.append((ylist[j] + velocities[j][1] * seconds) % room_size[1])
    return (new_xlist, new_ylist)

In [179]:
def plot_positions(input_filename, room_size, sec):
    (xlist, ylist, velocities) = read_input2(input_filename)
    (xlist, ylist) = positions_after_seconds(xlist, ylist, velocities, sec, room_size)
    
    fig, ax = plt.subplots()
    ax.plot(xlist, ylist, c='g', marker='s', ls='')
    ax.set_xlim(-1, room_size[0]+1)
    ax.set_ylim(room_size[1]+1, -1)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()

In [172]:
from PIL import Image, ImageFilter

def create_animation(input_filename, room_size, sec_start, sec_end, step_size):
    (xlist, ylist, velocities) = read_input2(input_filename)

    sec = sec_start
    if sec > 0:
        (xlist, ylist) = positions_after_seconds(xlist, ylist, velocities, sec, room_size)

    fig, ax = plt.subplots()
    sp, = ax.plot([], [], c='g', marker='s', ls='')
    ax.set_xlim(-1, room_size[0]+1)
    ax.set_ylim(room_size[1]+1, -1)
    ax.set_xticks([])
    ax.set_yticks([])

    while sec < sec_end:
        sp.set_data(xlist, ylist)
        ax.set_title(f'Seconds:{sec}')
        plt.savefig(f'../figures/anim_{input_filename}_{sec}')

        (xlist, ylist) = positions_after_seconds(xlist, ylist, velocities, step_size, room_size)
        sec += step_size

    images = []
    for sec in range(sec_start, sec_end, step_size):
        exec('img'+str(sec)+'=Image.open("'+f'../figures/anim_{input_filename}_{sec}'+'.png")')
        images.append(eval('img'+str(sec)))
    images[0].save(f'../figures/anim__{input_filename}_{sec_start}_{sec_end}_{step_size}.gif', save_all=True, append_images=images[1:], duration=500, loop=1)
